In [1]:
import os
import cv2
import numpy as np
from PIL import Image
import sys
import random
import zlib

In [2]:
BASE_PATH = "/home/varad/6422-DB-implementation/eva/eva/tutorials/blob-storage"
NPY_STORE_NAME = "video-blob.npy"
VIDEO_NUM = "12"
# ZLIB_COMPRESS_RATIO = 6
VIDEO_SRC = "/home/varad/6422-DB-implementation/eva/video/" + VIDEO_NUM + ".mp4"

NPY_STORE = BASE_PATH + "/" + NPY_STORE_NAME

In [3]:
# frame_offsets = {} # frame number: (offset, size)

In [4]:
if not os.path.exists(os.path.dirname(NPY_STORE)):
    try:
        os.makedirs(os.path.dirname(NPY_STORE))
    except OSError as exc:
        # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

In [5]:
cap = cv2.VideoCapture(VIDEO_SRC)
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [6]:
%%time
buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
sys.getsizeof(buf)

CPU times: user 59 µs, sys: 20 µs, total: 79 µs
Wall time: 82.7 µs


2307916944

In [7]:
# %%time
# mem_buf = np.empty((frameHeight, frameWidth, 3), np.dtype('uint8'))
# fc = 0
# ret = True
# offset = 0

# with open(NPY_STORE, 'ab') as fd:
#     while (fc < frameCount and ret):
#         ret, mem_buf = cap.read()
        
#         compressed_data = zlib.compress(mem_buf, ZLIB_COMPRESS_RATIO)
#         frame_offsets[fc] = (offset, sys.getsizeof(compressed_data))
        
#         offset += sys.getsizeof(compressed_data)
#         fd.write(compressed_data)
#         fc += 1
# cap.release()

In [8]:
%%time
# # Manual append yeilds problematic results, np.save stores metadata for better retrieval mechanism.
# # Should go with the recommended way to avoid compatibility issues further down the line
fc = 0
ret = True

while (fc < frameCount  and ret):
    ret, buf[fc] = cap.read()
    fc += 1

cap.release()

# # this is time consuming, can be optimized with memmap??
# # np memmaps for write are tricky: https://stackoverflow.com/questions/16149803/working-with-big-data-in-python-and-numpy-not-enough-ram-how-to-save-partial-r
with open(NPY_STORE, 'wb') as blobStore:
    np.save(blobStore, buf)

CPU times: user 5.48 s, sys: 2.58 s, total: 8.06 s
Wall time: 10.1 s


In [9]:
# %%time

# import zlib
# compressed_data = zlib.compress(buf, ZLIB_COMPRESS_RATIO)

# f = open("/home/varad/6422-DB-implementation/eva/eva/tutorials/blob-storage/" + VIDEO_NUM, 'wb')
# f.write(compressed_data)
# f.close()

In [10]:
def get_nth_frame(n):
    img = np.memmap(NPY_STORE, dtype='uint8', mode='r+', shape=(frameHeight, frameWidth, 3), offset=n * frameHeight * frameWidth * 3)
    return img

In [11]:
def get_range_of_frames(start_range, end_range):
    img_range = np.memmap(NPY_STORE, dtype='uint8', mode='r+', shape=(end_range - start_range + 1, frameHeight, frameWidth, 3), offset= start_range * frameHeight * frameWidth * 3)
    return img_range

In [12]:
def show_np_array_img(img):
    img = cv2.cvtColor(img[...,::-1].copy(), cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img, 'RGB')
    img.show()

In [13]:
# %%time
# Tests on a 16 second countdown video (500 * 1920 * 1020 * 3)
rand_frame = random.randint(0, frameCount)
img = get_nth_frame(rand_frame)
# show_np_array_img(img)
# Should show number 16 for n = 120

# FRAME_NO = 0
# with open(NPY_STORE, 'rb') as fd:
#     print(frame_offsets[FRAME_NO])
#     fd.seek(frame_offsets[FRAME_NO][0], 0)
#     compressed_img = fd.read(frame_offsets[FRAME_NO][1])
#     decompressed_img = zlib.decompress(compressed_img)

# # print(frame_offsets)
# nonshaped_img = np.frombuffer(decompressed_img)
# print(nonshaped_img.size)

In [14]:
%%time
rand_frame_start = random.randint(0, frameCount - 10)
rand_frame_end = rand_frame_start + 10
img_range = get_range_of_frames(rand_frame_start, rand_frame_end)
# for img in img_range:
#     show_np_array_img(img)
# Comment out to check if the number changes from 16 to 15 for start_range = 120 and end_range = 125

CPU times: user 531 µs, sys: 244 µs, total: 775 µs
Wall time: 1.62 ms


In [15]:
%%time

rand_frame_start = random.randint(0, frameCount - 20)
rand_frame_end = rand_frame_start + 20
img_range = get_range_of_frames(rand_frame_start, rand_frame_end)

CPU times: user 767 µs, sys: 353 µs, total: 1.12 ms
Wall time: 2.31 ms


In [16]:
%%time

rand_frame_start = random.randint(0, frameCount - 30)
rand_frame_end = rand_frame_start + 30
img_range = get_range_of_frames(rand_frame_start, rand_frame_end)

CPU times: user 0 ns, sys: 1.04 ms, total: 1.04 ms
Wall time: 2.38 ms


In [17]:
%%time

rand_frame_start = random.randint(0, frameCount - 40)
rand_frame_end = rand_frame_start + 40
img_range = get_range_of_frames(rand_frame_start, rand_frame_end)

CPU times: user 635 µs, sys: 293 µs, total: 928 µs
Wall time: 2.03 ms


In [18]:
%%time

rand_frame_start = random.randint(0, frameCount - 50)
rand_frame_end = rand_frame_start + 50
img_range = get_range_of_frames(rand_frame_start, rand_frame_end)

CPU times: user 0 ns, sys: 934 µs, total: 934 µs
Wall time: 2.43 ms


In [19]:
%%time

rand_frame_start = random.randint(0, frameCount - 60)
rand_frame_end = rand_frame_start + 60
img_range = get_range_of_frames(rand_frame_start, rand_frame_end)

CPU times: user 687 µs, sys: 319 µs, total: 1.01 ms
Wall time: 2.4 ms


In [20]:
%%time

rand_frame_start = random.randint(0, frameCount - 90)
rand_frame_end = rand_frame_start + 90
img_range = get_range_of_frames(rand_frame_start, rand_frame_end)

CPU times: user 0 ns, sys: 779 µs, total: 779 µs
Wall time: 2.07 ms


In [21]:
%%time

rand_frame_start = random.randint(0, frameCount - 120)
rand_frame_end = rand_frame_start + 120
img_range = get_range_of_frames(rand_frame_start, rand_frame_end)

CPU times: user 0 ns, sys: 1.07 ms, total: 1.07 ms
Wall time: 1.94 ms


In [22]:
%%time

rand_frame_start = random.randint(0, frameCount - 150)
rand_frame_end = rand_frame_start + 150
img_range = get_range_of_frames(rand_frame_start, rand_frame_end)

CPU times: user 0 ns, sys: 664 µs, total: 664 µs
Wall time: 1.66 ms


In [23]:
%%time

rand_frame_start = random.randint(0, frameCount - 200)
rand_frame_end = rand_frame_start + 200
img_range = get_range_of_frames(rand_frame_start, rand_frame_end)

CPU times: user 659 µs, sys: 250 µs, total: 909 µs
Wall time: 2.15 ms


In [24]:
# %%time
# _ = np.load(NPY_STORE)